In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.ion()  # To immediately show plots

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

plt.style.use("seaborn")  # Recommended

In [2]:
from poliastro.twobody.propagation import cowell
from numba import njit
r0 = [-2384.46, 5729.01, 3050.46] * u.km
v0 = [-7.36138, -2.98997, 1.64354] * u.km / u.s
initial = Orbit.from_vectors(Earth, r0, v0)
@njit
def accel(t0, state, k):
    """Constant acceleration al igned with the velocity. """
    v_vec = state[3:]
    norm_v = (v_vec * v_vec).sum() ** .5
    return 1e-5 * v_vec / norm_v

initial.propagate(3 * u.day, method=cowell, ad=accel)

18255 x 21848 km x 28.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.008 (TT)

In [3]:
from poliastro.core.perturbations import J2_perturbation
tof = (48.0 * u.h).to(u.s)
final = initial.propagate(tof, method=cowell, ad=J2_perturbation, J2=Earth.J2.value, R=Earth.R.to(u.km).value)

In [4]:
((final.raan - initial.raan) / tof).to(u.deg / u.h)

<Quantity -0.17232668 deg / h>

In [5]:
((final.argp - initial.argp) / tof).to(u.deg / u.h)

<Quantity 0.28220397 deg / h>